In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler

from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN

In [ ]:
n = 501
window = 64

x = np.linspace(0, 50, n)
y = np.sin(x)

plt.plot(x, y)
plt.plot(x[:window], y[:window])

In [ ]:
test_size = 50

data = pd.DataFrame(data=y, index=x, columns=["sine"])
train = data.iloc[:-test_size]
test = data.iloc[-(test_size + window):]

In [ ]:
scaler = MinMaxScaler()

scaler.fit(train)
train_scaled = scaler.transform(train)
test_scaled = scaler.transform(test)
y = test_scaled[-test_size:]

In [ ]:
plt.plot(range(0, n-test_size), train_scaled, linewidth=4)
plt.plot(range(n-test_size-window, n), test_scaled)

In [ ]:
# batch size, time step, features
train_generator = TimeseriesGenerator(
    data=train_scaled,
    targets=train_scaled,
    length=window,
    batch_size=1
)

test_generator = TimeseriesGenerator(
    data=test_scaled,
    targets=test_scaled,
    length=window,
    batch_size=1
)

In [ ]:
model = Sequential()

# input_shape = (num_timesteps, num_features)
model.add(SimpleRNN(window, input_shape=(window, 1)))

model.add(Dense(1))

In [ ]:
model.compile(
    optimizer="adam",
    loss="mse",
)

In [ ]:
model.summary()

In [ ]:
model.fit(x=train_generator, epochs=5)

In [ ]:
pd.DataFrame(model.history.history).plot()

In [ ]:
predictions = model.predict(test_generator).reshape(test_size)

In [ ]:
plt.plot(predictions)
plt.plot(y)

In [ ]:
plt.plot(range(0, n-test_size), train_scaled)
plt.plot(range(n-test_size, n), predictions, linewidth=3)
plt.plot(range(n-test_size, n), y)